In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
from datetime import datetime
import numpy as np
from requests_html import HTMLSession

In [2]:
main_link = 'https://www.kariera.gr/en/jobs?title=&page=0&limit=20'

In [3]:
main_link = 'https://www.kariera.gr/en/jobs?title=&page=0&limit=20'
headers = {'User-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}
r = requests.get(main_link, headers=headers)
r

<Response [200]>

## Main Idea
### scrape all main_link + sub_links where sub_links=data%20analyst,data%20scientist,data%20engineer

### Company, Job title, Content, Location and job occupation.

In [4]:
soup = BeautifulSoup(r.content, 'html.parser')

In [5]:
max_page = soup.find('li',class_='ant-pagination-next').previous_sibling.text
max_pages = int(max_page)

In [6]:
max_pages

191

### working out how to turn pages:

In [7]:
main_link = 'https://www.kariera.gr/en/jobs?title=&page=0&limit=20'

In [8]:
main_link.split('&')[-2][-1]

'0'

In [9]:
page_link_list=[]
for i in range(0,max_pages+1):
    page_link_list.append('https://www.kariera.gr/en/jobs?title=&page={}&limit=20'.format(i))

In [10]:
page_link_list

['https://www.kariera.gr/en/jobs?title=&page=0&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=1&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=2&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=3&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=4&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=5&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=6&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=7&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=8&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=9&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=10&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=11&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=12&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=13&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=14&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=15&limit=20',
 'https://www.kariera.gr/en/jobs?title=&page=16&limit=20',
 'https

In [11]:
# the'rows' for the for loop
section = soup.findAll('section',class_='ZVuAW2ak')

In [12]:
soup.findAll('section',class_='ZVuAW2ak')[1].find('a',class_='h6 oyOA_Z_a').text.strip()

'COGNITY'

## company names

In [13]:
soup.findAll('section',class_='ZVuAW2ak')[1].find('a',class_='h6 oyOA_Z_a').text.strip()

'COGNITY'

## job title

In [14]:
soup.findAll('section',class_='ZVuAW2ak')[0].find('a',class_='h4 tGc1KEdv').text

'Πωλητής/Πωλήτρια -Τμήμα Sales & Marketing'

## location

In [15]:
soup.findAll('section',class_='ZVuAW2ak')[0].find('span',class_='small-body-text').text

'Athina'

## job occupation

In [16]:
soup.findAll('section',class_='ZVuAW2ak')[0].findAll('span')[-3].text

'Full-time/Part-time'

# WITHOUT JOB CONTENT

In [17]:
df = pd.DataFrame()

In [18]:
df

""


In [19]:
for i in tqdm(page_link_list):
    link = i
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    sections = soup.findAll('section',class_='ZVuAW2ak')
    for section in sections:
        x = section.find('a',class_='h4 tGc1KEdv').text
        if ('Data Analyst' in x) | ('Data Scientist' in x) | ('Data Engineer' in x): # maybe x is case sensitive so safer to do a loop 
            try:
                company = section.find('a',class_='h6 oyOA_Z_a').text.strip()
            except:
                company = np.nan
            try:
                job_title = section.find('a',class_='h4 tGc1KEdv').text
            except:
                job_title = np.nan
            try:
                location = section.find('span',class_='small-body-text').text
            except:
                location = np.nan
            try:
                job_occupation = section.findAll('span')[-3].text
            except:
                job_occupation = np.nan
            temp = pd.DataFrame([{
                    'company' : company,
                    'job_title' : job_title,
                    'location': location,
                    'job_occupation' : job_occupation,
                    'date_scraped' : datetime.now().strftime("%Y-%m-%d %H:%M")
                }])
            df = pd.concat([df, temp], ignore_index=True)

  0%|          | 0/192 [00:00<?, ?it/s]

In [20]:
df

,company,job_title,location,job_occupation,date_scraped
0,Sani/Ikos Group,IT Data Analyst / Thessaloniki,Thessaloniki,Full-time,2022-11-13 10:28
1,WEBHELP HELLAS,Data Analyst for Airbnb,Athina,Full-time,2022-11-13 10:28
2,Mellon Technologies,Data Analyst,Pireas,Full-time,2022-11-13 10:28
3,Ernst and Young,Junior Data Engineer,Athina,Full-time,2022-11-13 10:29
4,Ernst and Young,Data & Analytics Manager - Data Engineering,Athina,Full-time,2022-11-13 10:29
5,Ernst and Young,Data Engineer - Patras,Patras,Full-time,2022-11-13 10:29
6,Ernst and Young,Data Scientist - Patras,Patras,Full-time,2022-11-13 10:29
7,Tech4sGroup,Data Analyst,Limassol,Full-time,2022-11-13 10:29
8,Celestino,Data Scientist - Business Intelligence Analyst,Athina,Full-time,2022-11-13 10:29
9,Netcompany-Intrasoft,Data Engineer,Athina,Full-time,2022-11-13 10:29


# WITH CONTENT

## getting all sub-links from all links

In [21]:
link_list=[]
session = HTMLSession()
for i in tqdm(page_link_list):
    for y in session.get(i).html.links:
        if '/en/jobs/' in y:
            link_list.append(y)

  0%|          | 0/192 [00:00<?, ?it/s]

In [22]:
len(link_list)

3807

In [23]:
final_list=[]
main = 'https://www.kariera.gr/'
for i in link_list:
    final_list.append(main + i)

In [24]:
test_link = final_list[2]

In [25]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}
r = requests.get(test_link, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="initial-scale=1.0, width=device-width" name="viewport"/><meta content="website" property="og:type"/><meta content="0kddlwbsdagaz58pjd2fj7p4wuuroo" name="facebook-domain-verification"/><link as="image" imagesrcset="/_next/image?url=%2Fassets%2Fimages%2Fflag-en.png&amp;w=32&amp;q=75 1x, /_next/image?url=%2Fassets%2Fimages%2Fflag-en.png&amp;w=48&amp;q=75 2x" rel="preload"/><title>Bilingual Customer Service Representative - Romanian-English - Travel - 32853</title><script type="application/ld+json">{"@context":"http://schema.org","@type":"JobPosting","datePosted":"2022-11-11T20:59:12.000Z","description":"<p><strong>What You’ll be Doing </strong><br />Looking to build your career in the travel industry? Whether it’s getting answers for customers quickly, consulting on products or resolving their issues with a smile, you’ll be the difference between their customer experience being just average or an exceptional one. 

### job title

In [26]:
soup.find('div',class_='h4 UGhAL2CE').text

'Bilingual Customer Service Representative - Romanian-English - Travel'

### company

In [27]:
soup.find('a',class_='__8_RJM1pE').text

'TTEC'

### location

In [28]:
soup.findAll('div',class_='DllOEHb_ main-body-text')[0].text

'Athina'

### job occupation

In [29]:
soup.findAll('div',class_='DllOEHb_ main-body-text')[-1].text

'Call Center / Customer Support'

### level

In [30]:
soup.findAll('div',class_='DllOEHb_ main-body-text')[2].text

'Entry/Junior'

### job type

In [31]:
soup.findAll('div',class_='DllOEHb_ main-body-text')[-2].text

'Full-time'

### content

In [32]:
result = soup.findAll('div',class_='__2DY4wJ3z hi8OBmAZ')
a=''
for res in result:
    a= (res.get_text(strip=True,separator=' '))
    a = a.replace('\u202f','')
a

"What You’ll be Doing Looking to build your career in the travel industry? Whether it’s getting answers for customers quickly, consulting on products or resolving their issues with a smile, you’ll be the difference between their customer experience being just average or an exceptional one. Turn your passion for travel into action in customer service helping people find great places to stay and creating unique travel experiences. During a Typical Day, You’ll Answer incoming communications from customers Conduct research to provide answers for customers to resolve their issues Be the first point of contact for the world's first-class accommodation partners by resolving basic questions and queries, such as modifications to rooms, rates, and availability, extranet tooling etc. Provide accurate, valid, and complete information by using the right tools, methods, and processes. Ensure a high level of partner service and a positive partner experience What You Bring to the Role Fluent in Romani

# building the loop

In [33]:
df2 = pd.DataFrame()

In [34]:
for i in tqdm(final_list):
    link = i
    a=''
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    x = soup.find('div',class_='h4 UGhAL2CE').text
    if ('Data Analyst' in x) | ('Data Scientist' in x) | ('Data Engineer' in x):
        try:
            job_title = soup.find('div',class_='h4 UGhAL2CE').text
        except:
            job_title = np.nan
        try:
            company = soup.find('a',class_='__8_RJM1pE').text
        except:
            company = np.nan
        try:
            location = soup.findAll('div',class_='DllOEHb_ main-body-text')[0].text
        except:
            location = np.nan
        try:
            job_occupation = soup.findAll('div',class_='DllOEHb_ main-body-text')[-1].text
        except:
            job_occupation = np.nan
        try:
            level = soup.findAll('div',class_='DllOEHb_ main-body-text')[2].text
        except:
            level = np.nan
        try:
            job_type = soup.findAll('div',class_='DllOEHb_ main-body-text')[-2].text
        except:
            job_type = np.nan
        try:
            result = soup.findAll('div',class_='__2DY4wJ3z hi8OBmAZ')
            for i in result:
                a= i.get_text(strip=True,separator=' ')
                a = a.replace('\u202f','')
        except:
            content = np.nan
        temp = pd.DataFrame([{
                'job_title' : job_title,
                'company' : company,
                'location': location,
                'job_occupation' : job_occupation,
                'level' : level,
                'job_type' : job_type,
                'content' : a,
                'date_scraped' : datetime.now().strftime("%Y-%m-%d %H:%M")
            }])
        df2 = pd.concat([df2, temp], ignore_index=True)

  0%|          | 0/3807 [00:00<?, ?it/s]

In [35]:
df2

,job_title,company,location,job_occupation,level,job_type,content,date_scraped
0,IT Data Analyst / Thessaloniki,Sani/Ikos Group,Thessaloniki,IT,Mid level,Full-time,Who do you want to become? Are you our Future ...,2022-11-13 10:33
1,Data Analyst for Airbnb,WEBHELP HELLAS,Athina,Part remote,Mid level,General Management,Requirements SQL skills (for optimally queryin...,2022-11-13 10:40
2,Data Analyst,Mellon Technologies,Pireas,Part remote,Mid level,IT,Description Responsible for the quality and co...,2022-11-13 10:45
3,Data Engineer - Patras,Ernst and Young,Patras,IT,Mid level,Full-time,"Requisition ID: 146677 At EY, you’ll have the ...",2022-11-13 10:48
4,Data & Analytics Manager - Data Engineering,Ernst and Young,Athina,IT,Senior,Full-time,"Requisition ID: 814902 At EY, you’ll have the ...",2022-11-13 10:49
5,Data Scientist - Patras,Ernst and Young,Patras,IT,Mid level,Full-time,"Requisition ID: 147180 At EY, you’ll have the ...",2022-11-13 10:49
6,Junior Data Engineer,Ernst and Young,Athina,IT,Mid level,Full-time,Requisition ID: 127061 Junior Data Engineer At...,2022-11-13 10:49
7,Data Analyst,Tech4sGroup,Limassol,BI / Business Analysis,Mid level,Full-time,"Tech4S Group, is a fast growing and innovative...",2022-11-13 10:49
8,Data Scientist - Business Intelligence Analyst,Celestino,Athina,IT,Entry/Junior,Full-time,Description We are looking for a Data Scientis...,2022-11-13 10:52
9,Data Engineer,Netcompany-Intrasoft,Athina,IT,Mid level,Full-time,Company Description We are Netcompany-Intrasof...,2022-11-13 10:56


In [36]:
df.to_csv('simple', index=False)

In [37]:
df2.to_csv('content', index=False)